# 🎨 NeMo Data Designer: Product Information Dataset Generator with Q&A using Open Router's Distillation endpoints API

#### 📚 What you'll learn

This notebook demonstrates how to use NeMo Data Designer to create a synthetic dataset of product information with corresponding questions and answers using OpenRouter's distillation endpoints

### Install the package if not installed

In [1]:
# !pip install data-designer==0.1.5
# !pip install dotenv==1.2.1

In [2]:
from dotenv import load_dotenv
import os
env = load_dotenv()

Open_Router_Api_Key = os.getenv('OPEN_ROUTER')
Base_URL = "https://openrouter.ai/api/v1"

### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [3]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    LLMTextColumnConfig,
    PersonSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    ModelProvider,
    UniformSamplerParams,
    Score,
    ModelConfig,
    InferenceParameters,
    ExpressionColumnConfig,
    LLMStructuredColumnConfig,
    BernoulliSamplerParams,
    LLMJudgeColumnConfig,
)

# If we want debugging - uncomment the following
# from data_designer.logging import configure_logging, LoggingConfig, OutputConfig
# import sys

# # Enable debug logging with structured JSON output to see full API call details
# configure_logging(
#     LoggingConfig(
#         logger_configs=[LoggingConfig.debug().logger_configs[0]],
#         output_configs=[OutputConfig(destination=sys.stderr, structured=True)],  # Use JSON format
#         root_level="DEBUG"
#     )
# )


### Try to call the OpenRouter endpoint first

In [4]:
from openai import OpenAI

client = OpenAI(
  base_url=Base_URL,
  api_key=Open_Router_Api_Key,
)

# First API call with reasoning
completion = client.chat.completions.create(
    model="nvidia/nemotron-3-nano-30b-a3b:free",
    messages=[{"role": "user", "content": "what is 1+1"}],
    max_tokens=100,
    stream=False,
    extra_body={"reasoning": {"enabled": False}}
)

reasoning = getattr(completion.choices[0].message, "reasoning", None)
if reasoning:
  print(reasoning)
  print('---')
print(completion.choices[0].message.content)


1 + 1 = 2.


### ⚙️ Initialize the NeMo Data Designer Client

- `NeMoDataDesignerClient` is responsible for submitting generation requests to the microservice.


In [5]:
model_provider = ModelProvider(
    name = "nvidia",
    endpoint = "https://openrouter.ai/api/v1/",
    provider_type = "openai",
    api_key = Open_Router_Api_Key,
    extra_body={
        "provider": {
            "enforce_distillable_text": True,
            # optionally, prefer NVIDIA endpoints
            "only": ["nvidia"]
        }
    }
)

data_designer_client = DataDesigner(model_providers=[model_provider])

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see [the model config docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/configure-models.html) for more details).


In [6]:

model_alias="nemotron-3-nano-30b-a3b"

inference_parameters = InferenceParameters(
    temperature=0.5,
    top_p=0.9,
    max_tokens=10000,
    max_parallel_requests=10,  # Number of concurrent workers
    extra_body={
        "reasoning": {"enabled": False}
    },
)

model_configs = [
    ModelConfig(
        alias=model_alias,
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        provider="nvidia",
        inference_parameters=inference_parameters
        )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [7]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏗️ Defining Data Structures

Now we'll define the data models and evaluation rubrics for our product information dataset.


In [8]:
import string
from pydantic import BaseModel
from pydantic import Field


# Define product information structure
class ProductInfo(BaseModel):
    product_name: str = Field(
        ..., description="A realistic product name for the market."
    )
    key_features: list[str] = Field(
        ..., min_length=1, max_length=3, description="Key product features."
    )
    description: str = Field(
        ...,
        description="A short, engaging description of what the product does, highlighting a unique but believable feature.",
    )
    price_usd: float = Field(..., description="The stated price in USD.")

## 🎲 Adding Sampler Columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.


In [9]:
# Define product category options
config_builder.add_column(
    SamplerColumnConfig(
        name="category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home Appliances",
                "Groceries",
                "Toiletries",
                "Sports Equipment",
                "Toys",
                "Books",
                "Pet Supplies",
                "Tools & Home Improvement",
                "Beauty",
                "Health & Wellness",
                "Outdoor Gear",
                "Automotive",
                "Jewelry",
                "Watches",
                "Office Supplies",
                "Gifts",
                "Arts & Crafts",
                "Baby & Kids",
                "Music",
                "Video Games",
                "Movies",
                "Software",
                "Tech Devices",
            ]
        ),
    )
)

# Define price range to seed realistic product types
config_builder.add_column(
    SamplerColumnConfig(
        name="price_tens_of_dollars",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(low=1, high=200),
    )
)

config_builder.add_column(
    ExpressionColumnConfig(
        name="product_price",
        expr="{{ (price_tens_of_dollars * 10) - 0.01 | round(2) }}",
        dtype="float",
    )
)

# Generate first letter for product name to ensure diversity
config_builder.add_column(
    SamplerColumnConfig(
        name="first_letter",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=list(string.ascii_uppercase)),
    )
)

# Determine if this example will include hallucination
config_builder.add_column(
    SamplerColumnConfig(
        name="is_hallucination",
        sampler_type=SamplerType.BERNOULLI,
        params=BernoulliSamplerParams(p=0.5),
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "category",
        "price_tens_of_dollars",
        "first_letter",
        "is_hallucination"
    ]
    expression_columns: ['product_price']
)

## 🦜 LLM-generated columns

- When prompting the LLM, we can use Jinja templating to reference other columns in the dataset.

- As we see below, nested json fields can be accessed using dot notation.


In [10]:
# Generate product information
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="product_info",
        model_alias=model_alias,
        prompt=(
            "Generate a realistic product description for a product in the {{ category }} "
            "category that costs {{ product_price }}.\n"
            "The name of the product MUST start with the letter {{ first_letter }}.\n"
        ),
        output_format=ProductInfo,
    )
)

# Generate user questions about the product
config_builder.add_column(
    LLMTextColumnConfig(
        name="question",
        model_alias=model_alias,
        prompt=("Ask a question about the following product:\n\n {{ product_info }}"),
    )
)


# Generate answers to the questions
config_builder.add_column(
    LLMTextColumnConfig(
        name="answer",
        model_alias=model_alias,
        prompt=(
            "{%- if is_hallucination == 0 -%}\n"
            "<product_info>\n"
            "{{ product_info }}\n"
            "</product_info>\n"
            "{%- endif -%}\n"
            "User Question: {{ question }}\n"
            "Directly and succinctly answer the user's question.\n"
            "{%- if is_hallucination == 1 -%}\n"
            "Make up whatever information you need to in order to answer the user's request.\n"
            "{%- endif -%}"
        ),
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "category",
        "price_tens_of_dollars",
        "first_letter",
        "is_hallucination"
    ]
    llm_text_columns: ['question', 'answer']
    llm_structured_columns: ['product_info']
    expression_columns: ['product_price']
)

## 🔍 Quality Assessment: LLM-as-a-Judge

When generating our synthetic dataset, we need to determine the quality of the generated data \
We use the LLM-as-a-Judge strategy to do this.

To do so, we need to define the rubric that the LLM should use to assess generation quality along with a prompt
that provides relavant instructions.


In [11]:
# Define evaluation rubrics for answer quality
CompletenessRubric = Score(
    name="Completeness",
    description="Evaluation of AI assistant's thoroughness in addressing all aspects of the user's query.",
    options={
        "Complete": "The response thoroughly covers all key points requested in the question, providing sufficient detail to satisfy the user's information needs.",
        "PartiallyComplete": "The response addresses the core question but omits certain important details or fails to elaborate on relevant aspects that were requested.",
        "Incomplete": "The response significantly lacks necessary information, missing major components of what was asked and leaving the query largely unanswered.",
    },
)

AccuracyRubric = Score(
    name="Accuracy",
    description="Evaluation of how factually correct the AI assistant's response is relative to the product information.",
    options={
        "Accurate": "The information provided aligns perfectly with the product specifications without introducing any misleading or incorrect details.",
        "PartiallyAccurate": "While some information is correctly stated, the response contains minor factual errors or potentially misleading statements about the product.",
        "Inaccurate": "The response presents significantly wrong information about the product, with claims that contradict the actual product details.",
    },
)


# Evaluate answer quality
config_builder.add_column(
    LLMJudgeColumnConfig(
        name="llm_answer_metrics",
        model_alias=model_alias,
        prompt=(
            "<product_info>\n"
            "{{ product_info }}\n"
            "</product_info>\n"
            "User Question: {{question }}\n"
            "AI Assistant Answer: {{ answer }}\n"
            "Judge the AI assistant's response to the user's question about the product described in <product_info>."
        ),
        scores=[CompletenessRubric, AccuracyRubric],
    )
)


# Extract metric scores for easier analysis
config_builder.add_column(
    ExpressionColumnConfig(
        name="completeness_result",
        expr="{{ llm_answer_metrics.completeness.score }}",
    )
)

config_builder.add_column(
    ExpressionColumnConfig(
        name="accuracy_result",
        expr="{{ llm_answer_metrics.accuracy.score }}",
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "category",
        "price_tens_of_dollars",
        "first_letter",
        "is_hallucination"
    ]
    llm_text_columns: ['question', 'answer']
    llm_structured_columns: ['product_info']
    llm_judge_columns: ['llm_answer_metrics']
    expression_columns: ['product_price', 'completeness_result', 'accuracy_result']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [12]:
# Preview a few records
preview = data_designer_client.preview(config_builder)

[08:36:47] [INFO] 🎥 Preview generation in progress
[08:36:47] [INFO] ✅ Validation passed
[08:36:47] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[08:36:47] [INFO] 🩺 Running health checks for models...
[08:36:48] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b:free' in provider named 'nvidia' for model alias 'nemotron-3-nano-30b-a3b'...
[08:36:48] [INFO]   |-- ✅ Passed!
[08:36:48] [INFO] 🎲 Preparing samplers to generate 10 records across 4 columns
[08:36:48] [INFO] 🧩 Generating column `product_price` from expression
[08:36:48] [INFO] 🗂️ Preparing llm-structured column generation
[08:36:48] [INFO]   |-- column name: 'product_info'
[08:36:48] [INFO]   |-- model config:
{
    "alias": "nemotron-3-nano-30b-a3b",
    "model": "nvidia/nemotron-3-nano-30b-a3b:free",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 0.9,
        "max_tokens": 10000,
        "max_parallel_requests": 10,
        "timeout": null,
        "extra_body": {
          

In [13]:
# More previews
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                  ┃ Value                                                                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ category              │ Clothing                                                                                │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ price_tens_of_dollars │ 175.92442528057435                                                                      │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ first_letter          │ C                                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ is_hallucination      │ 1                                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ product_price         │ 1759.2342528057436                                                                      │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ completeness_result   │ PartiallyComplete                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ accuracy_result       │ PartiallyAccurate                                                                       │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ question              │ What makes the Crestline Alpine Wool Blazer suitable for both urban and outdoor         │
│                       │ environments?                                                                           │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ answer                │ The Crestline Alpine Wool Blazer combines urban sophistication with rugged              │
│                       │ functionality through its premium wool blend fabric, which offers natural breathability │
│                       │ and moisture-wicking properties ideal for city wear while providing insulation against  │
│                       │ cooler mountain air. Its tailored silhouette ensures a polished look in professional or │
│                       │ social settings, while the fabric’s durability and wind-resistant finish allow seamless │
│                       │ transition to outdoor activities like hiking or skiing. Reinforced seams and a slightly │
│                       │ relaxed fit enhance comfort during movement, and the blazer’s minimalist                │
│                       │ design—featuring subtle earth tones and clean lines—avoids overt outdoor aesthetics,    │
│                       │ making it versatile for both settings. This balance of refined style and practical      │
│                       │ performance makes it a seamless choice for urban professionals and outdoor enthusiasts  │
│                       │ alike.                                                                                  │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ product_info          │ {                                                                                       │
│                       │     'product_name': 'Crestline

### View rows generated

In [14]:
final_dataset = preview.dataset.copy()
final_dataset.head()

,category,price_tens_of_dollars,first_letter,is_hallucination,product_price,product_info,question,answer,llm_answer_metrics,completeness_result,accuracy_result
0,Clothing,175.924425,C,1,1759.234253,{'product_name': 'Crestline Alpine Wool Blazer...,What makes the Crestline Alpine Wool Blazer su...,The Crestline Alpine Wool Blazer combines urba...,{'completeness': {'reasoning': 'The response a...,PartiallyComplete,PartiallyAccurate
1,Baby & Kids,13.352953,P,0,133.519534,"{'product_name': 'Padded Playtime Mattress', '...",What age range or weight limit is the Padded P...,The product description does not specify an ag...,{'completeness': {'reasoning': 'The AI assista...,PartiallyComplete,Accurate
2,Groceries,196.827308,M,1,1968.263082,{'product_name': 'MegaMuesli Organic Breakfast...,What makes the MegaMuesli Organic Breakfast Bl...,MegaMuesli Organic Breakfast Blend stands out ...,{'completeness': {'reasoning': 'The response a...,PartiallyComplete,PartiallyAccurate
3,Office Supplies,100.764655,S,1,1007.636552,"{'product_name': 'StaplerPro 5000', 'key_featu...",What specific office tasks or materials is the...,The StaplerPro 5000 is engineered for high‑vol...,{'completeness': {'reasoning': 'The response a...,PartiallyComplete,PartiallyAccurate
4,Arts & Crafts,102.580234,Q,0,1025.792341,{'product_name': 'Quill & Co. Watercolor Brush...,What makes the Quill & Co. Watercolor Brush Se...,The set’s hand‑crafted bamboo handles and ultr...,{'completeness': {'reasoning': 'The response a...,PartiallyComplete,Accurate


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [15]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 11                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                         ┃        data type ┃             number unique values ┃        sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ category                            │           string │                        8 (80.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ price_tens_of_dollars               │            float │                      10 (100.0%) │             uniform │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ first_letter                        │           string │                      10 (100.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ is_hallucination                    │              int │                        2 (20.0%) │           bernoulli │
└─────────────────────────────────────┴──────────────────┴──────────────────────────────────┴─────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                  ┃               ┃                              ┃       prompt tokens ┃       completion tokens ┃
┃ column name      ┃     data type ┃         number unique values ┃          per record ┃              per record ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ question         │        string │                  10 (100.0%) │       112.5 +/- 9.4 │            22.0 +/- 7.4 │
├──────────────────┼───────────────┼──────────────────────────────┼─────────────────────┼─────────────────────────┤
│ answer           │        string │                  10 (100.0%) │       61.0 +/- 51.8 │          112.0 +/- 95.8 │
└──────────────────┴───────────────┴──────────────────────────────┴─────────────────────┴─────────────────────────┘
                                                                                                                   
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [16]:
job_results = data_designer_client.create(config_builder, num_records=20)

[08:36:54] [INFO] 🎨 Creating Data Designer dataset
[08:36:54] [INFO] ✅ Validation passed
[08:36:54] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[08:36:54] [INFO] 🩺 Running health checks for models...
[08:36:54] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b:free' in provider named 'nvidia' for model alias 'nemotron-3-nano-30b-a3b'...
[08:36:55] [INFO]   |-- ✅ Passed!
[08:36:55] [INFO] ⏳ Processing batch 1 of 1
[08:36:55] [INFO] 🎲 Preparing samplers to generate 20 records across 4 columns
[08:36:55] [INFO] 🧩 Generating column `product_price` from expression
[08:36:55] [INFO] 🗂️ Preparing llm-structured column generation
[08:36:55] [INFO]   |-- column name: 'product_info'
[08:36:55] [INFO]   |-- model config:
{
    "alias": "nemotron-3-nano-30b-a3b",
    "model": "nvidia/nemotron-3-nano-30b-a3b:free",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 0.9,
        "max_tokens": 10000,
        "max_parallel_requests": 10,
        "timeou

### View the generated dataset

In [17]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,category,price_tens_of_dollars,first_letter,is_hallucination,product_price,product_info,question,answer,llm_answer_metrics,completeness_result,accuracy_result
0,Clothing,112.013238,Z,1,1120.122379,{'description': 'Experience luxurious comfort ...,What occasions or outfits do you think the adj...,The adjustable satin lapels on the **ZaraLux™ ...,{'accuracy': {'reasoning': 'All statements abo...,PartiallyComplete,Accurate
1,Baby & Kids,40.557527,Z,1,405.56527,{'description': 'The ZippyZoo Soft Play Mat of...,What age range or developmental stage is the Z...,The ZippyZoo Soft Play Mat is specifically des...,{'accuracy': {'reasoning': 'The stated age ran...,PartiallyComplete,PartiallyAccurate
2,Office Supplies,4.217897,M,0,42.16897,{'description': 'The MegaMemo Magnetic Whitebo...,What are the dimensions and weight of the Mega...,The provided product information does not incl...,{'accuracy': {'reasoning': 'The assistant accu...,PartiallyComplete,Accurate
3,Video Games,94.645561,R,0,946.445608,{'description': 'RogueStar Galaxy immerses pla...,What unique tactical advantages do the procedu...,The procedurally generated star systems with d...,{'accuracy': {'reasoning': 'The response corre...,PartiallyComplete,PartiallyAccurate
4,Baby & Kids,122.205552,L,0,1222.045517,{'description': 'The LumiGlow Nightlight gentl...,What color temperature options are available i...,The LumiGlow Nightlight offers three color‑tem...,{'accuracy': {'reasoning': 'The product descri...,PartiallyComplete,Inaccurate


### View the report

In [18]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 11                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                         ┃        data type ┃             number unique values ┃        sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ category                            │           string │                       13 (65.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ price_tens_of_dollars               │            float │                      20 (100.0%) │             uniform │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ first_letter                        │           string │                       14 (70.0%) │            category │
├─────────────────────────────────────┼──────────────────┼──────────────────────────────────┼─────────────────────┤
│ is_hallucination                    │              int │                        2 (10.0%) │           bernoulli │
└─────────────────────────────────────┴──────────────────┴──────────────────────────────────┴─────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                  ┃               ┃                              ┃        prompt tokens ┃      completion tokens ┃
┃ column name      ┃     data type ┃         number unique values ┃           per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ question         │        string │                  20 (100.0%) │       117.0 +/- 18.6 │           25.5 +/- 6.5 │
├──────────────────┼───────────────┼──────────────────────────────┼──────────────────────┼────────────────────────┤
│ answer           │        string │                  20 (100.0%) │       136.0 +/- 48.0 │         81.0 +/- 160.4 │
└──────────────────┴───────────────┴──────────────────────────────┴──────────────────────┴────────────────────────┘
                                                                                                                   
                                                          

### Output CSV to folder

In [19]:
from pathlib import Path

Folder_Name = "data-designer-tutorial-output"
File_Name = "dataset_OR.csv"

TUTORIAL_OUTPUT_PATH = Path(Folder_Name)
TUTORIAL_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

dataset.to_csv(TUTORIAL_OUTPUT_PATH / File_Name, index=False)